<a href="https://colab.research.google.com/github/ghrahul/ML-DL-PRACTICE/blob/master/Tensorflow%20in%20TPU/Tensorflow_CV_TPUipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From Srivatsan Srinivasan AI Engineering

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import os
import datetime

%load_ext tensorboard
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
#we see that only CPU. We have connect it through GRPC to access TPU
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]

In [4]:
x_train.shape

(60000, 28, 28)

In [0]:
#expansion for RGB channel to work with model
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [6]:
x_train.shape

(60000, 28, 28, 1)

In [0]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Conv2D(32, 5, padding="same", activation="relu", input_shape=(28, 28, 1)))
  model.add(tf.keras.layers.MaxPooling2D())
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Conv2D(64, 5, padding="same", activation="relu"))
  model.add(tf.keras.layers.MaxPooling2D())
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Conv2D(128, 5, padding="same", activation="relu"))
  model.add(tf.keras.layers.MaxPooling2D())
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Conv2D(256, 5, padding="same", activation="relu"))
  model.add(tf.keras.layers.MaxPooling2D())
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(512))
  model.add(tf.keras.layers.Activation("relu"))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation("softmax"))
  return model

In [8]:
!mkdir /tmp/logs

mkdir: cannot create directory ‘/tmp/logs’: File exists


In [0]:
import os

resolver = tf.contrib.cluster_resolver.TPUClusterResolver("grpc://" + os.environ['COLAB_TPU_ADDR'])
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

with strategy.scope():
  model = create_model()
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'],
  )
  logdir = os.path.join("/tmp/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, profile_batch=3)

  history=model.fit(
      x_train.astype(np.float32), y_train.astype(np.float32),
      epochs=20, steps_per_epoch=50, validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
      validation_freq=5,
      callbacks=[tensorboard_callback]
  )